In [ ]:
# this cell is just for display purposes, NOT necessary for your code
%config IPCompleter.use_jedi = False
# rediculous bug in Jedi it seems

# READ FIRST - `ppv_setup.ini`
Before proceeding, let's make sure that the ppv_setup.ini is properly configured:

Contents of '~/.config/ppv_setup.ini'

> [paths]  
> plate_dir = /home/user/obsdata/plates/test5  
>
> \# If you want your plate files in, e.g., /home/user/data/plates/0150XX  
> \# and /home/user/data/plates/0160XX, then edit the plate_dir line to read  
> \# plate_dir = /home/user/data/plates  
>
> [username]  
> sdss_org = username_at_utah  
> \# NO quotes  
> \# If you do NOT have an account at Utah, please get one at  
> \# https://wiki.sdss.org/display/DATA/Utah+Accounts  


You NEED to edit this file before using any further `ppv` functionality. Importing `ppv` will automatically look for this file in the `~/.config` directory.


## A first quick spin with `ppv`

Note that `ppv` will automagically download the files it needs using rsync as long as `ppv_setup.ini` is correctly configured. 

**NOTE** If needed, you will be prompted for your utah password. For now, the call is through the `apogee.sdss.org` server at Utah.


In [ ]:
from astropy.table import Table
import ppv  # the FIRST time you import ppv, the platePlans.par file is downloaded
# you can ignore the WARNING

In [ ]:
# good practice to get the latest plateplan and plug files from Utah
ppv.ppv.update_platefiles()

In [ ]:
# table of targets from v0.1 of targetDB  (read 'Targets' to see how to get this file; just demoing here)
planet_targetDB = Table.read('/home/jquark/projects/sdss5/ppv/data/raw/mwm_planet_0.1.0_targetdb.fits.gz')
planet_targetDB

In [ ]:
# Create targets object (see Targets section below for more information)
from ppv.targets import Targets

planet_targets = Targets(planet_targetDB)
# Check in the 'targets' section below for more info on how to make a targets object
# If you are using the targetDB files, it's as easy as above!
print(planet_targets)


In [ ]:
# Define a plate run using the correct string identifier
from ppv.groups import Platerun

plate_run_2020 = Platerun('2020.10.a.mwm-bhm')


In [ ]:
# Get available targets based on position. 
# These targets COULD have been observed in this plate run, i.e.,
# their position is withing 1.49 degrees of a field center belonging to this plate run.
# This takes ~30 seconds, it's loading all the plates/fields associated with the plate run.
# If this is your first time needed these plates,
# they will be downloaded and you will get a password prompt for apogee.sdss.org (Utah)

maybe_observed = planet_targets.available_in(plate_run_2020)
planet_targets.data[maybe_observed]

In [ ]:
# Get targets that were assigned a fiber in this platerun.
observed = planet_targets.assigned_in(plate_run_2020)
planet_targets.data[observed]

In [ ]:
# Get targets that were "available" but NOT assigned a fiber.
not_observed = planet_targets.not_assigned_in(plate_run_2020)
planet_targets.data[not_observed]

In [ ]:
# Get fiber information for the assigned targets (includes instrument and firstcarton)
planet_targets.assigned_info(plate_run_2020)

## A longer `ppv` Walkthrough
Please read though to get a better sense of how `ppv` works

### Convenient References

In [ ]:
# From the base package, you can access the full plate summary
ppv.ppv.allplate_summary

In [ ]:
# You can also get a list of all available plateruns (this is handy)
ppv.ppv.available_plateruns()

### Plate
- A plate holds all the relevant information of a plate, including all targets that are assigned fibers.

- This class (and the Field and Platerun classes) will automagically download the relevant plugHoles files (you don't even need to think about it). 



In [ ]:
# let's say I am interested in plate 15004. Let's look at the plate summary.
ppv.ppv.allplate_summary[ppv.ppv.allplate_summary['plateid'] == 15004]

In [ ]:
from ppv.plate import Plate

In [ ]:
# Let's load plate 15004 and automatically download the file if it's missing.

p15004 = Plate(15004)
print(p15004)  # show the plate center and ID

# It will take just a few seconds to download if needed.
# This will only need to happen once for every "plate batch"
# which is every ~50 plates (at least one platerun)

Viola!

In [ ]:
p15004.targets  # this is a table containing all the positions that are assigned fibers in the plate

### Field
- A field represents a FOV on the sky. It has a center position
- All fields contain at least one plate
- If you want to know about targets at a particular position in the sky, you should do so by Field; the Plates within a field could be checked for multiple observations.

In [ ]:
from ppv.groups import Field

In [ ]:
# Let's look at the information for plate 15004 again
ppv.ppv.allplate_summary[ppv.ppv.allplate_summary['plateid'] == 15004]

In [ ]:
# The field name for plate 15004 is 
field_name = ppv.ppv.allplate_summary['name'][ppv.ppv.allplate_summary['plateid'] == 15004][0]  # [0] indx just turns column into the data entry
print(field_name)

In [ ]:
example_field = Field(field_name)
print(example_field)

In [ ]:
# We can see the program that submitted the field center
example_field.programname

In [ ]:
# The plates associated with the field
example_field.plates
# Just the one plate for this field
# Note, this is the full Plates object

In [ ]:
# There is also a targets property, which contains all the target information for assigned fibers in this field
# If there was more than one plate for this field, this table would a concatenation of all plates.
example_field.targets  # Almost identical to the Plate.targets table, except for a column showing the field name.

### Platerun
- A Platerun is simply a collection of fields (as a field is a collection of plates).


In [ ]:
from ppv.groups import Platerun

In [ ]:
# Let's check available plateruns again
ppv.ppv.available_plateruns()

In [ ]:
platerun_08c = Platerun('2020.08.c.bhm-mwm')  

In [ ]:
# When you access the fields of a platerun, they get loaded from disk.
# This takes approximately 1 second per field on my laptop. You only need to do this ONCE per session.
print(f'{len(platerun_08c.fields)} fields in prun_c\n')
print(platerun_08c.fields)
# ~ 30 fields

In [ ]:
# other handy things
print(platerun_08c.fieldnames)  # just get the field names associated with the plate run.
print(platerun_08c.platesummary) # print the part of the allplate_summary that contains this plate run.

In [ ]:
# A huge table containing all targets assigned fibers in the plate run.
# This is akin to the "summary" table sent around during the very first plate run
platerun_08c.targets  

### Targets
- Contains the set of targets that you are interested in checking.
- Used as an interface with Plate, Field, and Platerun.

#### Easiest way to completely load targets
Use the outputs from Jos&eacute;  for targetDB 0.1, you can download a carton of targets at

https://data.sdss.org/sas/sdss5/target/development/0.1.0/   (requires standard SDSS password)

In [ ]:
# Let's look at the mwm_planet carton
from astropy.table import Table


planet_targetDB = Table.read('/home/jquark/projects/sdss5/ppv/data/raw/mwm_planet_0.1.0_targetdb.fits.gz')

# and the table
planet_targetDB

#### Targets object creation

In [ ]:
# To create the Targets object, just supply a table-like object (astropy table, numpy record array, or a dictionary)
# The data object needs to AT LEAST contain a catalogID array.
# If you want to use 'available_in', you need to supply the RA and Dec as well.
# Let's look at the call signature
Targets?

`colnames` is a list of column/field names that refer to, in order, the catalogID, RA, and Dec array.

The default is: 
> colnames = ['catalogid', 'ra', 'dec']  

These are the names of the columns containing the catalogID, RA, and Dec; respectively, withing the targetDB files.
If you use these carton files, you can just give the astropy table and you are good to go.

Otherwise, specify the catalogID, RA, and Dec columns/fields/keys as strings.

In [ ]:
# The rest are optional and can be kept via the 'ancillary' keyword.
# Note, you COULD make a Targets object with 3 arrays of the same length: RA, DEC, catalogID

planet_targets = Targets(planet_targetDB)

In [ ]:
planet_targets

#### Available and Assigned
- A target is 'available' if the target's position is in a particular field.

- A target is 'assigned' if a fiber was assigned to that target.


In [ ]:
# Let's check if any planet_targets are in the example field

print(planet_targets.available_in(example_field))   # this is a boolean array just stating if a target is available
planet_targets.data[planet_targets.available_in(example_field)]

# Viola! We get the 11 targets that COULD have been observed

In [ ]:
# Now let's check what was assigned in this field
# Same approach as 'available' his is a boolean array just stating if a target is available
planet_targets.data[planet_targets.assigned_in(example_field)]

In [ ]:
# Only 8 of a possible 11 targets were observed! 
# Which ones were left out?!

planet_targets.data[planet_targets.not_assigned_in(example_field)]  # NOT_assigned_in

# BOOM!

- `available_in` and `assigned_in` will work for Plate objects, too. For example, we would have gotten the same results if we had done,\
\
`planet_targets.data[planet_targets.not_assigned_in(p15004)]`

- Better still, the same concepts (and call signature!) are available for plate runs!

In [ ]:
# Available
planet_targets.data[planet_targets.available_in(platerun_08c)]

In [ ]:
# Assigned_in works exactly the same way as for fields
planet_targets.data[planet_targets.assigned_in(platerun_08c)]

#### Getting more information for assigned targets
`available_in` and `assigned_in` help you understand things about your input Targets object.

To know HOW, exactly, your targets were assigned, use `assigned_info`.
This will pull the relevant rows from the plate files.

In [ ]:
planet_targets.assigned_info(example_field)
# information includes the instrument used ('holetype') and the carton the object was targeted under ('firstcarton')

In [ ]:
# also works for a full plate run!
planet_targets.assigned_info(platerun_08c)

Thanks for reading this through!